In [ ]:
!pip install --upgrade --quiet  langchain langchain-community langchain-groq langgraph

In [ ]:
import os
import json
from typing import TypedDict
from google.colab import userdata
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults


In [ ]:
#LLM set up
groq_api_key=userdata.get('GROQ_API')
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x797eda5c4490>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x797ed9977c50>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
#SEARCH SET UP
from langchain_community.tools.tavily_search import TavilySearchResults
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')
#tool = TavilySearchResults(max_results=3) #increased number of results
##print(type(tool))
#print(tool.name)

In [ ]:
class ChatState(TypedDict):
    messages: list
    query: str

@tool
def query_fact_check_websites(query: str):
    """
    Return a list of articles published on pesacheck.org and africacheck.org
    related to a user's question. If no articles are found, return a message
    indicating that.
    """
    print(f"Searching for articles related to: {query} on pesacheck.org and africacheck.org")

    search_tool = TavilySearchResults(max_results=5) # Increased max_results for potentially more results

    # Search Pesacheck
    pesacheck_results = search_tool.invoke(f"{query} site:pesacheck.org")
    pesacheck_articles = [{"source": "Pesacheck.org", "title": res['title'], "url": res['url'], "content": res['content']} for res in pesacheck_results]

    # Search Africacheck
    africacheck_results = search_tool.invoke(f"{query} site:africacheck.org")
    africacheck_articles = [{"source": "Africacheck.org", "title": res['title'], "url": res['url'], "content": res['content']} for res in africacheck_results]

    all_articles = pesacheck_articles + africacheck_articles

    if not all_articles:
        return "No articles found on pesacheck.org or africacheck.org related to your query."
    else:
        # Format the results nicely
        formatted_results = "Found the following articles:\n\n"
        for article in all_articles:
            formatted_results += f"Source: {article['source']}\n"
            formatted_results += f"Title: {article['title']}\n"
            formatted_results += f"URL: {article['url']}\n"
            formatted_results += f"Content Snippet: {article['content'][:200]}...\n\n" # Provide a content snippet

        return formatted_results


In [ ]:
# Define the graph
workflow = StateGraph(ChatState)

#Add nodes
# Add an LLM node to process the user input and potentially generate tool calls
workflow.add_node("llm", lambda state: llm.invoke(state["messages"]))
workflow.add_node("search", ToolNode([query_fact_check_websites]))

# Define the entry point
workflow.set_entry_point("llm")

# Define the exit point
# The LLM can either call the search tool or finish
workflow.add_edge("llm", "search") # Directly go to search after LLM for this simple example

# Define the exit point for the search tool
workflow.add_edge("search", END)


# Compile the graph
app = workflow.compile()

In [ ]:
results = app.invoke({"messages": [{"role": "user", "content": "Find articles on pesacheck.org and africacheck.org about Coffee is Uganda's biggest export."}]})
print(results)


ValueError: No AIMessage found in input

In [ ]:
# Add necessary imports if not already present
import os
import json
from typing import TypedDict, Annotated # Import Annotated here
from google.colab import userdata
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AIMessage # Import AIMessage
from langchain_core.messages import HumanMessage # Import HumanMessage
import operator # Import the operator module


# Ensure necessary libraries are installed
!pip install --upgrade --quiet  langchain langchain-community langchain-groq langgraph beautifulsoup4 requests

# LLM set up
groq_api_key = userdata.get('GROQ_API')
llm = ChatGroq(groq_api_key=groq_api_key, model_name="Gemma2-9b-It")

# SEARCH SET UP
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

class ChatState(TypedDict):
    # Use Annotated and operator.add to manage messages
    messages: Annotated[list, operator.add]
    # Keep query for potential future use, but it's not strictly needed for the current tool signature
    query: str


@tool
def query_fact_check_websites(query: str):
    """
    Return a list of articles published on pesacheck.org and africacheck.org
    related to a user's question. If no articles are found, return a message
    indicating that.
    """
    print(f"Searching for articles related to: {query} on pesacheck.org and africacheck.org")

    search_tool = TavilySearchResults(max_results=5)

    # Search Pesacheck
    pesacheck_results = search_tool.invoke(f"{query} site:pesacheck.org")
    pesacheck_articles = [{"source": "Pesacheck.org", "title": res['title'], "url": res['url'], "content": res['content']} for res in pesacheck_results]

    # Search Africacheck
    africacheck_results = search_tool.invoke(f"{query} site:africacheck.org")
    africacheck_articles = [{"source": "Africacheck.org", "title": res['title'], "url": res['url'], "content": res['content']} for res in africacheck_results]

    all_articles = pesacheck_articles + africacheck_articles

    if not all_articles:
        return "No articles found on pesacheck.org or africacheck.org related to your query."
    else:
        formatted_results = "Found the following articles:\n\n"
        for article in all_articles:
            formatted_results += f"Source: {article['source']}\n"
            formatted_results += f"Title: {article['title']}\n"
            formatted_results += f"URL: {article['url']}\n"
            formatted_results += f"Content Snippet: {article['content'][:200]}...\n\n"

        return formatted_results

# Define the graph
workflow = StateGraph(ChatState)

# Add nodes
# Add an LLM node. We need to bind the tool to the LLM so it knows how to call it.
# The LLM's output will be added to the state's messages.
workflow.add_node("llm", lambda state: llm.bind_tools([query_fact_check_websites]).invoke(state["messages"]))
# The ToolNode will execute the tool called by the LLM.
workflow.add_node("search", ToolNode([query_fact_check_websites]))

# Define the entry point
workflow.set_entry_point("llm")

# Define the conditional edge from the LLM node
# The LLM can either call a tool or finish
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # If the last message is an AIMessage with tool_calls, then we should go to the search node
    if isinstance(last_message, AIMessage) and hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "search"
    else:
        # Otherwise, the LLM has finished, so we should end
        return END

workflow.add_conditional_edges(
    "llm", # Start node
    should_continue, # Conditional logic
    {
        "search": "search", # If should_continue returns "search", go to the search node
        END: END # If should_continue returns END, end the graph
    }
)

# Define the edge from the search node back to the LLM
# After the search tool executes, its results are added to the state, and the LLM should process them.
workflow.add_edge("search", "llm")


# Compile the graph
app = workflow.compile()

# Example usage:
results = app.invoke({"messages": [HumanMessage(content="Find articles on pesacheck.org and africacheck.org about Coffee is Uganda's biggest export.")]})
print(results)

{'messages': [HumanMessage(content="Find articles on pesacheck.org and africacheck.org about Coffee is Uganda's biggest export.", additional_kwargs={}, response_metadata={})]}
